In [47]:
import sys 
import os
import subprocess
import numpy as np 
import time
from itertools import product 
import glob
import re
import pandas as pd
import pysynapse
from ddot import Ontology
import multiprocessing as mp
from itertools import chain


output_dir = '../output/analyzing_clixo_ontologies'; 
if not os.path.exists(output_dir): os.makedirs(output_dir)

In [48]:
def get_alpha_beta(l):
    """
    Yield function to get the correct alpha and beta names.
    
    """
    for x in l: 
        alpha = '{:.1f}'.format(x[0])
        beta =  '{:.1f}'.format(x[1])
        yield((alpha, beta))
        
    yield(None, None)

def read_clixo_stats(fn):
    dic = {'num_valid_clusters': np.nan, 'largest_cluster': np.nan, 
          'num_edges_in_clustergraph': np.nan, 'num_clusters': np.nan}
    with open(fn) as f: 
        for line in f: 
            if line.startswith('#'):
                if 'Num valid clusters' in line: 
                    dic['num_valid_clusters'] = int(line.strip().split()[-1])
                elif 'Largest cluster' in line:
                    dic['largest_cluster'] = int(line.strip().split()[-1])
                elif 'Num edges in clusterGraph' in line:
                    dic['num_edges_in_clustergraph'] = int(line.strip().split()[-1])
                elif 'Num clusters' in line:
                    dic['num_clusters'] = int(line.strip().split()[-1])

    return(dic)

## Clean the CliXo ontology for Enrichment Analysis 

In [49]:
def generate_clean_ont_file(fn, out_fn): 
    
    with open(fn) as fr, open(out_fn, 'w') as fw: 
        
        fw.write('Parent\tChild\tEdgeType\n')
        for line in fr:
            if line.startswith('#'):
                continue
            else:
                line = line.strip().split('\t')
                line[2] = line[2].replace('default', 'Child-Parent')
                line[2] = line[2].replace('gene', 'Gene-Term')
                line = '\t'.join(line[0:3]) + '\n'
                fw.write(line)
    return(clean_fn)

In [64]:
pattern = 'option([0-9])'
for fn in sorted(glob.glob('../output/run_clixo/option*/*')):
    option = re.search(pattern, fn).groups()[0]
    clean_fn = 'option{}_{}'.format(option, os.path.basename(fn))
    clean_fn = os.path.join(output_dir, clean_fn)
    if not os.path.exists(clean_fn):
        clean_fn = generate_clean_ont_file(fn, clean_fn) 

## Calculate the number of genes 

In [51]:
def read_number_of_genes(fn):
    genes = set() 
    with open(fn) as f:
        for line in f:
            line = line.split() 
            if line[2] == 'Gene-Term':
                genes.add(line[1]) 
    return(len(genes))

In [52]:
gene_stats = []
pattern = 'option([0-9]).*alpha([0-9]*\.[0-9]*)_beta([0-9]*\.[0-9]*)'
for fn in sorted(glob.glob(os.path.join(output_dir, '*txt'))):
        
    params = re.search(pattern, fn)
    option, alpha, beta = [float(x) for x in params.groups()]  
    
    clean_fn = os.path.basename(fn)
    clean_fn = os.path.join(output_dir, clean_fn)
    
    num_genes = read_number_of_genes(clean_fn)
    gene_stats.append([option, alpha, beta, num_genes])

In [53]:
gene_stats = pd.DataFrame(gene_stats, columns=['option', 'alpha', 'beta', 'num_genes'])
option_grps = gene_stats.groupby('option')

In [59]:
gene_stats[gene_stats.num_genes == 0 ]

,option,alpha,beta,num_genes
70,4.0,0.6,0.5,0
75,4.0,0.7,0.5,0
80,4.0,0.8,0.5,0
85,4.0,0.9,0.5,0
86,4.0,0.9,0.6,0


In [54]:
option_grps.num_genes.describe()

,count,mean,std,min,25%,50%,75%,max
option,,,,,,,,
2.0,45.0,1283.000000,0.00000,1283.0,1283.0,1283.0,1283.0,1283.0
4.0,45.0,1493.333333,533.93905,0.0,1680.0,1680.0,1680.0,1680.0


In [55]:
option_grps.num_genes.value_counts()

option  num_genes
2.0     1283         45
4.0     1680         40
        0             5
Name: num_genes, dtype: int64

In [56]:
option_grps.get_group(4.0).tail(7)

,option,alpha,beta,num_genes
83,4.0,0.8,0.8,1680
84,4.0,0.8,0.9,1680
85,4.0,0.9,0.5,0
86,4.0,0.9,0.6,0
87,4.0,0.9,0.7,1680
88,4.0,0.9,0.8,1680
89,4.0,0.9,0.9,1680


In [57]:
option_grps.get_group(2.0).tail(7)

,option,alpha,beta,num_genes
38,2.0,0.8,0.8,1283
39,2.0,0.8,0.9,1283
40,2.0,0.9,0.5,1283
41,2.0,0.9,0.6,1283
42,2.0,0.9,0.7,1283
43,2.0,0.9,0.8,1283
44,2.0,0.9,0.9,1283


## Calculate CliXo statistics 

In [72]:
clixo_stats = {}
pattern = 'option([0-9]).*alpha([0-9]*\.[0-9]*)_beta([0-9]*\.[0-9]*)'
for fn in sorted(glob.glob('../output/run_clixo/option*/*')):
        
    params = re.search(pattern, fn)
    option, alpha, beta = [float(x) for x in params.groups()] 
    option = int(option)
    if alpha == None:
        break

    # Getting CliXo stats 
    stats = read_clixo_stats(fn) 
    
    clean_fn = 'option{}_{}'.format(option, os.path.basename(fn))
    clean_fn = os.path.join(output_dir, clean_fn)
    
    # Getting GO enrichment stats
    with open(clean_fn) as f: 
        file_len = len(f.readlines())
    if file_len > 1: 
        go_enrichment = pysynapse.compare_to_go(clean_fn)
    else:
        go_enrichment = np.nan
    stats['go_enrichment'] = go_enrichment    
    key = tuple([option, alpha, beta]) 
    clixo_stats[key] = stats

In [73]:
clixo_stats_df = pd.DataFrame.from_dict(clixo_stats, orient='index')
clixo_stats_df.reset_index(inplace=True)
clixo_stats_df.rename(columns={'level_0': 'option', 
                               'level_1': 'alpha',
                               'level_2': 'beta'}, inplace=True)

In [74]:
clixo_stats_df.go_enrichment.describe()

count    85.000000
mean     43.000000
std      19.396244
min      10.000000
25%      28.000000
50%      41.000000
75%      52.000000
max      86.000000
Name: go_enrichment, dtype: float64

## Analyze disease enrichment with Multiprocessing 

In [75]:
def analyze_gene_enrichment_test(ont_fn, disease_gene_fn, output):

    ## We read in our ontology
    #Generate custom ontology from Chromatin branch from human GO
    #ontology_file=Generate_Ontology_File('GO:0000785')
    ont = Ontology.from_table(ont_fn)
    translated = pysynapse.Find_GO_Focus_GeneDict(ont)

    #Test genes: autism
    text_file = open(disease_gene_fn, "r")
    test_gene_list = text_file.read().splitlines()
    text_file.close()

    #print("Number of autism genes in our ontology:" ,  len(set(ont.genes).intersection(set(test_gene_list))))
    num_ont_disease_genes = len(set(ont.genes).intersection(set(test_gene_list)))

    #Find number of test genes in enriched modules:
    num_enriched_disease_genes = pysynapse.Find_num_genes_in_enriched(ont, translated, test_gene_list)

    output.put((num_ont_disease_genes, num_enriched_disease_genes))

In [76]:
diseases = ['adhd', 'autism', 'bipolar', 'mdd', 'schizophrenia']

In [90]:
disease_stats = {}

# Define an output queue
output = mp.Queue()

pattern = 'option([0-9]).*alpha([0-9]*\.[0-9]*)_beta([0-9]*\.[0-9]*)'
clean_fns = glob.glob(os.path.join(output_dir + '/*.txt'))
for clean_fn in sorted(clean_fns):
          
    params = re.search(pattern, clean_fn)
    option, alpha, beta = [float(x) for x in params.groups()]    
    
    with open(clean_fn) as f: 
        file_len = len(f.readlines())
        
    if file_len > 1: 
        # Setup a list of processes that we want to run
        processes = []
        for disease in diseases: 
            disease_gene_fn = '../output/omim_psychiatric_disease_genes/{}.txt'.format(disease)
            p = mp.Process(target=analyze_gene_enrichment_test, 
                           args=(clean_fn, disease_gene_fn, output))
            processes.append(p)

        # Run processes
        for p in processes:
            p.start()

        # Exit the completed processes
        for p in processes:
            p.join()

        # Get process results from the output queue
        results = [output.get() for p in processes]

        disease_stats[(option, alpha, beta)] = results 
        time.sleep(0.1)

2.0 0.1 0.5
2.0 0.1 0.6
2.0 0.1 0.7
2.0 0.1 0.8
2.0 0.1 0.9
2.0 0.2 0.5
2.0 0.2 0.6
2.0 0.2 0.7
2.0 0.2 0.8
2.0 0.2 0.9
2.0 0.3 0.5
2.0 0.3 0.6
2.0 0.3 0.7
2.0 0.3 0.8
2.0 0.3 0.9
2.0 0.4 0.5
2.0 0.4 0.6
2.0 0.4 0.7
2.0 0.4 0.8
2.0 0.4 0.9
2.0 0.5 0.5
2.0 0.5 0.6
2.0 0.5 0.7
2.0 0.5 0.8
2.0 0.5 0.9
2.0 0.6 0.5
2.0 0.6 0.6
2.0 0.6 0.7
2.0 0.6 0.8
2.0 0.6 0.9
2.0 0.7 0.5
2.0 0.7 0.6
2.0 0.7 0.7
2.0 0.7 0.8
2.0 0.7 0.9
2.0 0.8 0.5
2.0 0.8 0.6
2.0 0.8 0.7
2.0 0.8 0.8
2.0 0.8 0.9
2.0 0.9 0.5
2.0 0.9 0.6
2.0 0.9 0.7
2.0 0.9 0.8
2.0 0.9 0.9
4.0 0.1 0.5
4.0 0.1 0.6
4.0 0.1 0.7
4.0 0.1 0.8
4.0 0.1 0.9
4.0 0.2 0.5
4.0 0.2 0.6
4.0 0.2 0.7
4.0 0.2 0.8
4.0 0.2 0.9
4.0 0.3 0.5
4.0 0.3 0.6
4.0 0.3 0.7
4.0 0.3 0.8
4.0 0.3 0.9
4.0 0.4 0.5
4.0 0.4 0.6
4.0 0.4 0.7
4.0 0.4 0.8
4.0 0.4 0.9
4.0 0.5 0.5
4.0 0.5 0.6
4.0 0.5 0.7
4.0 0.5 0.8
4.0 0.5 0.9
4.0 0.6 0.6
4.0 0.6 0.7
4.0 0.6 0.8
4.0 0.6 0.9
4.0 0.7 0.6
4.0 0.7 0.7
4.0 0.7 0.8
4.0 0.7 0.9
4.0 0.8 0.6
4.0 0.8 0.7
4.0 0.8 0.8
4.0 0.8 0.9
4.0 0.9 0.7
4.0 

In [92]:
disease_cols = [('{}_genes'.format(disease), 
                 '{}_enriched_genes'.format(disease)) for disease in diseases]
disease_cols = list(chain(*disease_cols))
disease_cols = ['option', 'alpha', 'beta'] + disease_cols
disease_stats_tmp = [list(k) + list(chain(*v)) for k, v in disease_stats.items()]
disease_stats_df = pd.DataFrame(disease_stats_tmp, columns=disease_cols)

## Merge the dataframes together 

In [99]:
merged_df = pd.merge(clixo_stats_df, disease_stats_df, 
                     on=['option', 'alpha', 'beta'], how='left')
fn = os.path.join(output_dir, 'ontology_stats.tsv')
merged_df.to_csv(fn, sep='\t', index=False)